# EDA

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

# Train set
df_train = pd.read_csv("train_5_norm_region.csv")
df_train["RESALE_PRICE"] = np.log(df_train["RESALE_PRICE"])

print("===== Data Overview =====")
print(df_train.info(), "\n")

for col in df_train.columns:
    print(f"--- Column: {col} ---")
    dtype = df_train[col].dtype
    print(f"Data Type: {dtype}")

    missing_count = df_train[col].isnull().sum()
    print(f"# Missing Values: {missing_count}")
    
    if np.issubdtype(dtype, np.number):
        col_data = df_train[col].dropna()
        print("Type: Continuous (numeric)")
        print(f"Min: {col_data.min()}")
        print(f"Max: {col_data.max()}")
        print(f"Mean: {col_data.mean():.2f}")
        print(f"Std: {col_data.std():.2f}")
    else:   
        unique_values = df_train[col].dropna().unique()
        print("Type: Categorical")
        print(f"Unique Values Count: {len(unique_values)}")
        print(f"Unique Values Set: {set(unique_values)}")

    print("\n")


===== Data Overview =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162691 entries, 0 to 162690
Data columns (total 44 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   MONTH                     162691 non-null  object 
 1   TOWN                      162691 non-null  object 
 2   FLAT_TYPE                 162691 non-null  object 
 3   BLOCK                     162691 non-null  object 
 4   STREET                    162691 non-null  object 
 5   FLOOR_RANGE               162691 non-null  object 
 6   FLOOR_AREA_SQM            162691 non-null  float64
 7   FLAT_MODEL                162691 non-null  object 
 8   ECO_CATEGORY              162691 non-null  object 
 9   LEASE_COMMENCE_DATA       162691 non-null  int64  
 10  RESALE_PRICE              162691 non-null  float64
 11  address                   162691 non-null  object 
 12  LATITUDE                  162691 non-null  float64
 13  LONGITUDE         

In [46]:
# Test set
df_test = pd.read_csv("test_5_norm_region.csv")

print("===== Data Overview =====")
print(df_test.info(), "\n")

for col in df_test.columns:
    print(f"--- Column: {col} ---")
    dtype = df_test[col].dtype
    print(f"Data Type: {dtype}")

    missing_count = df_test[col].isnull().sum()
    print(f"# Missing Values: {missing_count}")
    
    if np.issubdtype(dtype, np.number):
        col_data = df_test[col].dropna()
        print("Type: Continuous (numeric)")
        print(f"Min: {col_data.min()}")
        print(f"Max: {col_data.max()}")
        print(f"Mean: {col_data.mean():.2f}")
        print(f"Std: {col_data.std():.2f}")
    else:
        unique_values = df_test[col].dropna().unique()
        print("Type: Categorical")
        print(f"Unique Values Count: {len(unique_values)}")
        print(f"Unique Values Set: {set(unique_values)}")

    print("\n")

===== Data Overview =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   MONTH                     50000 non-null  object 
 1   TOWN                      50000 non-null  object 
 2   FLAT_TYPE                 50000 non-null  object 
 3   BLOCK                     50000 non-null  object 
 4   STREET                    50000 non-null  object 
 5   FLOOR_RANGE               50000 non-null  object 
 6   FLOOR_AREA_SQM            50000 non-null  float64
 7   FLAT_MODEL                50000 non-null  object 
 8   ECO_CATEGORY              50000 non-null  object 
 9   LEASE_COMMENCE_DATA       50000 non-null  int64  
 10  address                   50000 non-null  object 
 11  LATITUDE                  50000 non-null  float64
 12  LONGITUDE                 50000 non-null  float64
 13  DIST_PRIMARY_1            50000 non

## Processing "TOWN" Column

In [47]:
# Target Encoding for TOWN

town_mean_map = df_train.groupby("TOWN")["RESALE_PRICE"].mean().to_dict()

df_train["TOWN_TARGET_ENC"] = df_train["TOWN"].map(town_mean_map)
df_test["TOWN_TARGET_ENC"]  = df_test["TOWN"].map(town_mean_map)

global_mean = df_train["RESALE_PRICE"].mean()
df_test["TOWN_TARGET_ENC"] = df_test["TOWN_TARGET_ENC"].fillna(global_mean)

print("Target encoding applied on 'TOWN'.")
print(df_train[["TOWN_TARGET_ENC"]].head())
print(df_test[["TOWN_TARGET_ENC"]].head())

✅ Target encoding applied on 'TOWN'.
   TOWN_TARGET_ENC
0        13.018201
1        13.409189
2        13.072957
3        13.166479
4        13.099824
   TOWN_TARGET_ENC
0        13.073720
1        13.275217
2        13.409189
3        13.099824
4        13.072957


## Processing "FLAT_TYPE" Column 

In [48]:
# One-Hot Encoding for "FLAT_TYPE" column
from sklearn.preprocessing import OneHotEncoder

df_train["FLAT_TYPE"] = (
    df_train["FLAT_TYPE"].astype(str).str.replace("-", " ", regex=False)
)

flat_type_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
flat_train = flat_type_encoder.fit_transform(df_train[["FLAT_TYPE"]])

flat_cols = flat_type_encoder.get_feature_names_out(["FLAT_TYPE"])
flat_train_df = pd.DataFrame(flat_train, columns=flat_cols, index=df_train.index)
df_train = pd.concat([df_train, flat_train_df], axis=1)

print(df_train.head())
print(set(df_train["FLAT_TYPE"].unique()))

     MONTH           TOWN FLAT_TYPE BLOCK              STREET FLOOR_RANGE  \
0  2020-10      woodlands    4 room  681B  woodlands drive 62    07 to 09   
1  2021-07         bishan    4 room   264    bishan street 24    07 to 09   
2  2021-05  bukit panjang    4 room   520       jelapang road    19 to 21   
3  2021-08        punggol    4 room  121B     edgedale plains    16 to 18   
4  2023-05        hougang    5 room  997B   Buangkok Crescent    10 to 12   

   FLOOR_AREA_SQM         FLAT_MODEL   ECO_CATEGORY  LEASE_COMMENCE_DATA  ...  \
0           102.0  premium apartment  uncategorized                 2000  ...   
1           104.0            model a  uncategorized                 1992  ...   
2           102.0            model a  uncategorized                 1998  ...   
3            93.0            model a  uncategorized                 2017  ...   
4           113.0           improved  uncategorized                 2018  ...   

   REGION_north-east region REGION_west region  TO

In [49]:
# Apply the same transformation to the test set
df_test["FLAT_TYPE"] = (
    df_test["FLAT_TYPE"].astype(str).str.replace("-", " ", regex=False)
)

flat_test = flat_type_encoder.fit_transform(df_test[["FLAT_TYPE"]])

flat_test_df = pd.DataFrame(flat_test, columns=flat_cols, index=df_test.index)
df_test = pd.concat([df_test, flat_test_df], axis=1)

print(df_test.head())
print(set(df_test["FLAT_TYPE"].unique()))

     MONTH           TOWN FLAT_TYPE BLOCK            STREET FLOOR_RANGE  \
0  2025-06      sembawang    3 room  115D     Canberra Walk    04 to 06   
1  2020-01    bukit merah    4 room  118A     jalan membina    01 to 03   
2  2025-06         bishan    3 room    22     Sin Ming Road    10 to 12   
3  2022-10        hougang    4 room   635  Hougang Avenue 8    10 to 12   
4  2024-02  bukit panjang    4 room   275      bangkit road    10 to 12   

   FLOOR_AREA_SQM  FLAT_MODEL   ECO_CATEGORY  LEASE_COMMENCE_DATA  ...  \
0            68.0     model a  uncategorized                 2018  ...   
1            90.0     model a  uncategorized                 2003  ...   
2            68.0    improved  uncategorized                 1973  ...   
3            84.0  simplified  uncategorized                 1987  ...   
4           104.0     model a  uncategorized                 1988  ...   

  REGION_north-east region  REGION_west region  TOWN_TARGET_ENC  \
0                        0           

## Processing "MONTH" Column

In [50]:
df_train["YEAR_INT"] = df_train["MONTH"].str[:4].astype(int)
df_train["MONTH_INT"] = df_train["MONTH"].str[-2:].astype(int)

print(df_train[["MONTH", "YEAR_INT", "MONTH_INT"]].head())

     MONTH  YEAR_INT  MONTH_INT
0  2020-10      2020         10
1  2021-07      2021          7
2  2021-05      2021          5
3  2021-08      2021          8
4  2023-05      2023          5


In [51]:
df_test["YEAR_INT"] = df_test["MONTH"].str[:4].astype(int)
df_test["MONTH_INT"] = df_test["MONTH"].str[-2:].astype(int)

print(df_test[["MONTH", "YEAR_INT", "MONTH_INT"]].head())

     MONTH  YEAR_INT  MONTH_INT
0  2025-06      2025          6
1  2020-01      2020          1
2  2025-06      2025          6
3  2022-10      2022         10
4  2024-02      2024          2


## Processing "FLAT_MODEL" Column

In [52]:
def te_cv_train(df, col="FLAT_MODEL", target_col="RESALE_PRICE",
                n_splits=5, out_col=None, random_state=0):
    df = df.copy()
    if out_col is None:
        out_col = f"{col}_TARGET_ENC"
    df[out_col] = pd.NA

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for tr_idx, va_idx in kf.split(df):
        tr = df.iloc[tr_idx]
        va = df.iloc[va_idx]

        mean_map = tr.groupby(col)[target_col].mean().to_dict()
        s = df.loc[va.index, col].map(mean_map)
        fold_global = tr[target_col].mean()
        s = s.fillna(fold_global)

        df.loc[va.index, out_col] = s

    df[out_col] = df[out_col].astype(float)
    df[out_col] = df[out_col].fillna(df[target_col].mean())

    return df

df_train = te_cv_train(df_train, col="FLAT_MODEL", target_col="RESALE_PRICE", n_splits=5, out_col="FLAT_MODEL_TARGET_ENC")
print(df_train[["FLAT_MODEL", "FLAT_MODEL_TARGET_ENC"]].head())

          FLAT_MODEL  FLAT_MODEL_TARGET_ENC
0  premium apartment              13.201660
1            model a              13.096374
2            model a              13.096896
3            model a              13.097875
4           improved              13.115737


In [53]:
def build_te_mapping(df_train, col="FLAT_MODEL", target_col="RESALE_PRICE"):
    mean_map = df_train.groupby(col)[target_col].mean().to_dict()
    global_mean = df_train[target_col].mean()
    return mean_map, global_mean

def te_apply_test(df_test, mean_map, global_mean, col="FLAT_MODEL", out_col="FLAT_MODEL_TARGET_ENC"):
    df = df_test.copy()
    s = df[col].map(mean_map).fillna(global_mean)
    df[out_col] = s.astype(float)
    return df

mean_map, global_mean = build_te_mapping(df_train, col="FLAT_MODEL", target_col="RESALE_PRICE")
df_test = te_apply_test(df_test, mean_map, global_mean, col="FLAT_MODEL", out_col="FLAT_MODEL_TARGET_ENC")

print(df_test[["FLAT_MODEL", "FLAT_MODEL_TARGET_ENC"]].head())

   FLAT_MODEL  FLAT_MODEL_TARGET_ENC
0     model a              13.097340
1     model a              13.097340
2    improved              13.116094
3  simplified              12.872831
4     model a              13.097340


## Processing "FLOOR_RANGE" Column

In [54]:
import re
from bisect import bisect_left

_rng = re.compile(r'(\d+)\s*to\s*(\d+)', flags=re.IGNORECASE)

def parse_floor_range(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    m = _rng.search(s)
    if not m:
        return None
    lo, hi = int(m.group(1)), int(m.group(2))
    if lo > hi:
        lo, hi = hi, lo
    return lo, hi

def build_floor_order_mapping(df_train, col="FLOOR_RANGE"):
    pairs = (
        df_train[col]
        .dropna()
        .map(parse_floor_range)
        .dropna()
        .unique()
    )
    pairs = sorted(pairs, key=lambda t: (t[0], t[1]))
    mapping = {f"{lo:02d} to {hi:02d}": i for i, (lo, hi) in enumerate(pairs)}
    for i, (lo, hi) in enumerate(pairs):
        mapping[f"{lo} to {hi}"] = i

    lows = [lo for (lo, _) in pairs]
    median_code = int(np.median(list(mapping.values()))) if mapping else 0
    return mapping, pairs, lows, median_code

def encode_floor_range_series(sr, mapping, pairs, lows, median_code):
    enc = sr.map(mapping)
    mask_na = enc.isna()
    if mask_na.any():
        parsed = sr[mask_na].map(parse_floor_range)

        def fallback_code(p):
            if p is None:
                return median_code
            lo, hi = p
            if not lows:
                return median_code
            pos = bisect_left(lows, lo)
            pos = max(0, min(pos, len(pairs)-1))
            return pos

        enc.loc[mask_na] = parsed.map(fallback_code)

    return enc.astype(int)


mapping, pairs, lows, median_code = build_floor_order_mapping(df_train, col="FLOOR_RANGE")


df_train["FLOOR_RANGE_ORD"] = encode_floor_range_series(
    df_train["FLOOR_RANGE"], mapping, pairs, lows, median_code
)


df_test["FLOOR_RANGE_ORD"] = encode_floor_range_series(
    df_test["FLOOR_RANGE"], mapping, pairs, lows, median_code
)

print(df_train[["FLOOR_RANGE", "FLOOR_RANGE_ORD"]].head())
print(df_test[["FLOOR_RANGE", "FLOOR_RANGE_ORD"]].head())


  FLOOR_RANGE  FLOOR_RANGE_ORD
0    07 to 09                2
1    07 to 09                2
2    19 to 21                6
3    16 to 18                5
4    10 to 12                3
  FLOOR_RANGE  FLOOR_RANGE_ORD
0    04 to 06                1
1    01 to 03                0
2    10 to 12                3
3    10 to 12                3
4    10 to 12                3


## Drop Columns

In [55]:
# cols_to_drop = [
#     "MONTH", "TOWN", "FLAT_TYPE", "BLOCK",
#     "STREET", "FLOOR_RANGE", "FLAT_MODEL", "ECO_CATEGORY", "address",
#     "LONGITUDE", "LATITUDE"
# ]
cols_to_drop = [
    "MONTH", "TOWN", "FLAT_TYPE", "BLOCK",
    "STREET", "FLOOR_RANGE", "FLAT_MODEL", "ECO_CATEGORY", "address",
    "LONGITUDE", "LATITUDE",
    "DIST_MRT_MIN", "DIST_PRIMARY_SCHOOL_MIN", "DIST_SECONDARY_SCHOOL_MIN", "DIST_MALL_MIN", "DIST_HAWKER_MIN"
]
df_train = df_train.drop(columns=cols_to_drop, errors="ignore")
df_test = df_test.drop(columns=cols_to_drop, errors="ignore")

In [56]:
print(len(df_train.columns))
print(len(df_test.columns))
print(df_train.head())

45
44
   FLOOR_AREA_SQM  LEASE_COMMENCE_DATA  RESALE_PRICE  DIST_PRIMARY_1  \
0           102.0                 2000     12.948010        0.964544   
1           104.0                 1992     13.279367        0.838661   
2           102.0                 1998     13.017003        0.954103   
3            93.0                 2017     13.049793        0.955166   
4           113.0                 2018     13.473020        0.764329   

   DIST_PRIMARY_2  DIST_PRIMARY_3  DIST_PRIMARY_4  DIST_PRIMARY_5  \
0        0.893327        0.835897        0.898973        0.878608   
1        0.791415        0.768995        0.813526        0.825383   
2        0.952762        0.854563        0.900289        0.936102   
3        0.900274        0.914964        0.915332        0.935776   
4        0.800775        0.803989        0.869039        0.901083   

   DIST_SECONDARY_1  DIST_SECONDARY_2  ...  FLAT_TYPE_2 room  \
0          0.929884          0.822332  ...               0.0   
1          0.88107

# Train & Test

In [68]:
# ==========================================================
# CV + Hyperparam Search with tqdm, export CSV submissions
# Models: XGBoost / LightGBM
# ==========================================================
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


TARGET_COL = "RESALE_PRICE"
features = [c for c in df_train.columns if c != TARGET_COL]
X = df_train[features]
y = df_train[TARGET_COL]
X_test = df_test[features]


def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def cv_search(model_name, model_ctor, param_grid, X, y, n_splits=5, random_state=42):
    import os

    best_params, best_rmse = None, float("inf")
    best_eval_result = None
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    grid = list(ParameterGrid(param_grid))
    pbar = tqdm(grid, desc=f"{model_name} grid", leave=True)

    for params in pbar:
        fold_rmses = []
        eval_result = None

        for fold, (tr_idx, va_idx) in enumerate(kf.split(X)):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

            model = model_ctor(**params)

            if model_name.lower().startswith("xgboost"):
                model.set_params(eval_metric="rmse")
                model.fit(
                    X_tr, y_tr,
                    eval_set=[(X_tr, y_tr), (X_va, y_va)],
                    verbose=False
                )
                eval_result = model.evals_result()
            elif model_name.lower().startswith("lightgbm"):
                model.set_params(metric="rmse")
                model.fit(
                    X_tr, y_tr,
                    eval_set=[(X_tr, y_tr), (X_va, y_va)],
                )
                eval_result = model.evals_result_
            else:
                model.fit(X_tr, y_tr)

            pred = model.predict(X_va)
            rmse_real = rmse(np.exp(y_va), np.exp(pred))
            fold_rmses.append(rmse_real)

        mean_rmse = float(np.mean(fold_rmses))

        if mean_rmse < best_rmse:
            best_rmse, best_params = mean_rmse, params
            best_eval_result = eval_result
            print(f"\nNew best for {model_name}: RMSE={best_rmse:.0f}")
            for k, v in best_params.items():
                print(f"  {k}: {v}")
            print()

        short_params = {k: best_params[k] for k in ["max_depth", "learning_rate", "n_estimators"] if k in best_params}
        pbar.set_postfix({
            "cv_rmse": f"{mean_rmse:.0f}",
            "best": f"{best_rmse:.0f}",
            **short_params
        })

    os.makedirs("logs", exist_ok=True)
    if best_eval_result is not None:
        if model_name.lower().startswith("xgboost"):
            df_log = pd.DataFrame({
                "train_rmse": best_eval_result["validation_0"]["rmse"],
                "valid_rmse": best_eval_result["validation_1"]["rmse"]
            })
        elif model_name.lower().startswith("lightgbm"):
            df_log = pd.DataFrame({
                "train_rmse": best_eval_result["training"]["rmse"],
                "valid_rmse": best_eval_result["valid_1"]["rmse"]
            })
        else:
            df_log = pd.DataFrame()

        log_path = f"logs/{model_name.lower()}_best_train_log.csv"
        df_log.to_csv(log_path, index=False)
        print(f"Saved {model_name} best training log to {log_path}")

    print(f"\n===== {model_name} Best Result =====")
    print(f"CV RMSE (real price): {best_rmse:.0f}")
    print("Best Params:")
    for k, v in best_params.items():
        print(f"  {k}: {v}")
    print("=" * 40 + "\n")

    return best_params, best_rmse



def train_full_and_predict(model_ctor, params, X, y, X_test):
    model = model_ctor(**params)
    model.fit(X, y)
    pred_log = model.predict(X_test)
    pred_price = np.exp(pred_log)
    return pred_price

def save_submission(preds, path):
    sub = pd.DataFrame({"Id": np.arange(len(preds)), "Predicted": preds})
    sub.to_csv(path, index=False)
    return sub


xgb_grid = {
    "n_estimators": [2200, 2500],
    "learning_rate": [0.03, 0.025],
    "max_depth": [8],
    "subsample": [0.8],
    "colsample_bytree": [0.8],
    "reg_lambda": [1.0, 2.0],
    "random_state": [42],
    "n_jobs": [-1],
}
# xgb_grid = {
#     "n_estimators": [1600],
#     "learning_rate": [0.06],
#     "max_depth": [8],
#     "subsample": [0.8],
#     "colsample_bytree": [0.8],
#     "reg_lambda": [2.0],
#     "random_state": [42],
#     "n_jobs": [-1],
# }
lgbm_grid = {
    "n_estimators": [1500, 2000],
    "learning_rate": [0.05, 0.1],
    "max_depth": [8],
    "subsample": [0.8],
    "colsample_bytree": [0.8],
    "reg_lambda": [1.0, 3.0],
    "random_state": [42],
    "n_jobs": [-1],
}
# lgbm_grid = {
#     "n_estimators": [2000],
#     "learning_rate": [0.1],
#     "max_depth": [8],
#     "subsample": [0.8],
#     "colsample_bytree": [0.8],
#     "reg_lambda": [1.0],
#     "random_state": [42],
#     "n_jobs": [-1],
# }


xgb_best_params, xgb_cv = cv_search("XGBoost", XGBRegressor, xgb_grid, X, y)
lgbm_best_params, lgbm_cv = cv_search("LightGBM", LGBMRegressor, lgbm_grid, X, y)


xgb_preds  = train_full_and_predict(XGBRegressor, xgb_best_params, X, y, X_test)
lgbm_preds = train_full_and_predict(LGBMRegressor, lgbm_best_params, X, y, X_test)


save_submission(xgb_preds,  "xgb_submission.csv")
save_submission(lgbm_preds, "lgbm_submission.csv")


cv_scores = {
    "XGBoost": xgb_cv,
    "LightGBM": lgbm_cv
}
best_name = min(cv_scores, key=cv_scores.get)
print(f"\n Best by CV RMSE: {best_name} ({cv_scores[best_name]:.2f})")

best_preds = {"XGBoost": xgb_preds, "LightGBM": lgbm_preds}[best_name]
best_preds = {"XGBoost": xgb_preds}[best_name]
save_submission(best_preds, "submission_best.csv")

print("\nFiles saved:")
print(" - xgb_submission.csv")
print(" - lgbm_submission.csv")
print(" - submission_best.csv (best by CV)")


XGBoost grid:  12%|█▎        | 1/8 [01:25<10:00, 85.72s/it, cv_rmse=25412, best=25412, max_depth=8, learning_rate=0.03, n_estimators=2200]


New best for XGBoost: RMSE=25412
  colsample_bytree: 0.8
  learning_rate: 0.03
  max_depth: 8
  n_estimators: 2200
  n_jobs: -1
  random_state: 42
  reg_lambda: 1.0
  subsample: 0.8



XGBoost grid:  25%|██▌       | 2/8 [02:48<08:22, 83.82s/it, cv_rmse=25371, best=25371, max_depth=8, learning_rate=0.03, n_estimators=2200]


New best for XGBoost: RMSE=25371
  colsample_bytree: 0.8
  learning_rate: 0.03
  max_depth: 8
  n_estimators: 2200
  n_jobs: -1
  random_state: 42
  reg_lambda: 2.0
  subsample: 0.8



XGBoost grid:  75%|███████▌  | 6/8 [08:40<02:52, 86.01s/it, cv_rmse=25359, best=25359, max_depth=8, learning_rate=0.025, n_estimators=2200]

...

LightGBM grid:  88%|████████▊ | 7/8 [02:09<00:18, 18.74s/it, cv_rmse=25822, best=25822, max_depth=8, learning_rate=0.1, n_estimators=2000]


New best for LightGBM: RMSE=25822
  colsample_bytree: 0.8
  learning_rate: 0.1
  max_depth: 8
  n_estimators: 2000
  n_jobs: -1
  random_state: 42
  reg_lambda: 1.0
  subsample: 0.8

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005134 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6783
[LightGBM] [Info] Number of data points in the train set: 130152, number of used features: 44
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 13.099517
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007375 seconds.
You can set `force_col_wise=true` to remove the overh

LightGBM grid: 100%|██████████| 8/8 [02:33<00:00, 19.19s/it, cv_rmse=25785, best=25785, max_depth=8, learning_rate=0.1, n_estimators=2000]


New best for LightGBM: RMSE=25785
  colsample_bytree: 0.8
  learning_rate: 0.1
  max_depth: 8
  n_estimators: 2000
  n_jobs: -1
  random_state: 42
  reg_lambda: 3.0
  subsample: 0.8

Saved LightGBM best training log to logs/lightgbm_best_train_log.csv

===== LightGBM Best Result =====
CV RMSE (real price): 25785
Best Params:
  colsample_bytree: 0.8
  learning_rate: 0.1
  max_depth: 8
  n_estimators: 2000
  n_jobs: -1
  random_state: 42
  reg_lambda: 3.0
  subsample: 0.8



[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007989 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6784
[LightGBM] [Info] Number of data points in the train set: 162691, number of used features: 44
[LightGBM] [Info] Start training from score 13.099892
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

 Best by CV RMSE: XGBoost (25343.58)

Files saved:
 - xgb_submission.csv
 - lgbm_submission.csv
 - submission_best.csv (best by CV)


In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# === 路径配置 ===
log_dir = "logs"
xgb_log_path = os.path.join(log_dir, "xgboost_best_train_log.csv")
lgb_log_path = os.path.join(log_dir, "lightgbm_best_train_log.csv")
output_dir = "plots"
os.makedirs(output_dir, exist_ok=True)


def plot_single_model(df, model_name, save_path):
    plt.figure(figsize=(8, 5))
    plt.plot(df["train_rmse"], label=f"{model_name} Train RMSE", linestyle="--")
    plt.plot(df["valid_rmse"], label=f"{model_name} Valid RMSE", linewidth=2)
    plt.title(f"{model_name} Training vs Validation RMSE", fontsize=14)
    plt.xlabel("Iterations (Boosting Rounds)", fontsize=12)
    plt.ylabel("RMSE", fontsize=12)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Saved {model_name} plot to {save_path}")


if os.path.exists(xgb_log_path):
    xgb_log = pd.read_csv(xgb_log_path)
    plot_single_model(xgb_log, "XGBoost", os.path.join(output_dir, "xgboost_rmse_curve.png"))

if os.path.exists(lgb_log_path):
    lgb_log = pd.read_csv(lgb_log_path)
    plot_single_model(lgb_log, "LightGBM", os.path.join(output_dir, "lightgbm_rmse_curve.png"))

plt.figure(figsize=(10, 6))

if os.path.exists(xgb_log_path):
    plt.plot(xgb_log["train_rmse"], label="XGBoost Train RMSE", linestyle="--")
    plt.plot(xgb_log["valid_rmse"], label="XGBoost Valid RMSE", linewidth=2)

if os.path.exists(lgb_log_path):
    plt.plot(lgb_log["train_rmse"], label="LightGBM Train RMSE", linestyle="--")
    plt.plot(lgb_log["valid_rmse"], label="LightGBM Valid RMSE", linewidth=2)

plt.title("XGBoost vs LightGBM - Training vs Validation RMSE", fontsize=14)
plt.xlabel("Iterations (Boosting Rounds)", fontsize=12)
plt.ylabel("RMSE", fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()

compare_path = os.path.join(output_dir, "compare_xgb_vs_lgbm_rmse.png")
plt.savefig(compare_path, dpi=300)
plt.close()
print(f"Saved comparison plot to {compare_path}")


Saved XGBoost plot to plots\xgboost_rmse_curve.png
Saved LightGBM plot to plots\lightgbm_rmse_curve.png
Saved comparison plot to plots\compare_xgb_vs_lgbm_rmse.png
